In [6]:
import os
# os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import pandas as pd

In [7]:
import tensorflow as tf

TypeError: Expected a message Descriptor, got Descriptor

In [4]:
gpus=tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

NameError: name 'tf' is not defined

In [5]:
directory="D:/Course/DrivenData Project/Dataset/"
df=pd.read_csv(directory+'train_features.csv')
df

NameError: name 'pd' is not defined

In [ ]:
import numpy as np
train_labels = pd.read_csv(directory+'train_labels.csv')
classes = train_labels.columns[1:]
train_labels = train_labels.iloc[:, 1:]
train_y = np.array([np.argmax(train_labels.iloc[i, :]) for i in range(train_labels.shape[0])])
l=list(train_y)
l

In [ ]:
df['label']=train_y
df

In [ ]:
filepaths=df['filepath'].values
labels=df['label'].values
ds_train=tf.data.Dataset.from_tensor_slices((filepaths,labels))
len(ds_train)

In [ ]:
def read_image(image_file,label):
    image=tf.io.read_file(directory+image_file)
    image=tf.image.decode_image(image,channels=1,dtype=tf.float32,expand_animations = False)
    return image,label

In [ ]:
def resize_and_rescale(image,label):
    image=tf.cast(image,tf.float32)
    image=tf.image.resize(image,[224,224])
    return image,label

In [ ]:
import tensorflow_io as tfio
def grey(image,label):
#     image=tfio.experimental.color.bgr_to_rgb(image)
    image=(image/255.0)
    return image,label

In [ ]:
ds_train=ds_train.map(read_image).map(resize_and_rescale).map(grey).batch(32)
len(ds_train)

In [ ]:
for i in ds_train:
    print (i)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l1, l2, l1_l2

In [ ]:
model=Sequential()
model.add(Conv2D(60,(3,3),input_shape=(224,224,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(60,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(100,activation='relu',activity_regularizer=l1(0.001)))
model.add(Dense(1))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam' ,metrics=['accuracy'])

In [ ]:
model.fit(ds_train,epochs=5,verbose=1)